In [287]:
import os
import numpy as np
from itertools import product
from scipy.signal import fftconvolve, convolve

# Convolutions

In [18]:
dat2 = np.arange(4*4*4).reshape(4,4,4)

In [6]:
dat = np.arange(1,17, dtype=float).reshape(4,4)
kernel = np.ones((2,2), dtype=float)

In [23]:
np.round(np.arange(3, dtype=float)).astype(int)

array([0, 1, 2])

In [11]:
np.pad(dat,0,mode='constant')

array([[  1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.],
       [  9.,  10.,  11.,  12.],
       [ 13.,  14.,  15.,  16.]])

In [17]:
for i in product(range(3),range(1,4)):
    print i

(0, 1)
(0, 2)
(0, 3)
(1, 1)
(1, 2)
(1, 3)
(2, 1)
(2, 2)
(2, 3)


In [55]:
np.unravel_index(0, (2,2))

(0, 0)

In [60]:
np.ravel([[1],[1]], (2,2))

array([1, 1])

In [62]:
dat[(1,0)]

5.0

In [63]:
for i in product(*[np.arange(3) for i in range(2)]):
    print i
    

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)


In [72]:
dat[:2,:2]

array([[ 1.,  2.],
       [ 5.,  6.]])

In [70]:
dat[(slice(0,2), slice(0,2))]

array([[ 1.,  2.],
       [ 5.,  6.]])

In [73]:
tuple(i for i in range(3))

(0, 1, 2)

In [88]:
stride = 2

In [89]:
for x in product(*[stride*np.arange(i) for i in (2,2)]):
    print x

(0, 0)
(0, 2)
(2, 0)
(2, 2)


In [216]:
def my_convd(dat, kernel, stride=1, pad=0):
    """ Perform a dot-product convolution of a n-dim kernel with n-dim data"""
    assert stride > 0
    assert dat.ndim == kernel.ndim
    stride = int(round(stride))
    
    outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*pad)/stride+1.
    for num in outshape:
        assert (num).is_integer(), num
    outshape = np.round(outshape).astype(int)
    
    if pad:
        dat = np.pad(dat, pad, mode='constant')

    out = np.zeros(outshape, dtype='float64')
    indices = range(dat.ndim)
    for i,x in enumerate(product(*[stride*np.arange(i) for i in outshape])):
        slices = tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x))
        loc = np.unravel_index(i, outshape)
        out[loc] = np.einsum(dat[slices], indices, kernel, indices)

    return out

NameError: name 'concatenate' is not defined

In [441]:
def multi_layer_conv(data, kernel, stride, pad):
    dat = data[0]
    assert stride > 0
    assert dat.ndim == kernel.ndim
    stride = int(round(stride))
    
    outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*pad)/float(stride)+1.
    for num in outshape:
        assert (num).is_integer(), num
    outshape = np.round(outshape).astype(int)
    
    total_out_shape = tuple([data.shape[0]]+[i for i in outshape])
    total_out = np.zeros(total_out_shape, dtype=data.dtype)
    
    indices = range(dat.ndim)
    all_pos = list(product(*[stride*np.arange(i) for i in outshape]))
    all_slices = [tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x)) for x in all_pos]
    
    for n, dat in enumerate(data):
        if pad:
            dat = np.pad(dat, pad, mode='constant')
            out = np.zeros(outshape, dtype=data.dtype)
        for i,x in enumerate(all_pos):
            slices = all_slices[i]
            loc = np.unravel_index(i, outshape)
            out[loc] = np.einsum(dat[slices], indices, kernel, indices)
        total_out[n] = out
    return total_out

In [407]:
dat = np.arange(np.product((3,3,3))).reshape((3,3,3))

In [445]:
dat = np.arange(1,17, dtype=float).reshape(4,4)
# dat = np.arange(np.product((3,3,3))).reshape((3,3,3))
kernel = np.ones((1,1), dtype=float)
# kernel = np.ones((2,2,2), dtype=float)
stride = 1

pad = 1

In [446]:
big_dat = np.concatenate([dat,2*dat]).reshape(2,4,4)

In [447]:
multi_layer_conv(big_dat, kernel, stride, pad)

array([[[  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   1.,   2.,   3.,   4.,   0.],
        [  0.,   5.,   6.,   7.,   8.,   0.],
        [  0.,   9.,  10.,  11.,  12.,   0.],
        [  0.,  13.,  14.,  15.,  16.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   2.,   4.,   6.,   8.,   0.],
        [  0.,  10.,  12.,  14.,  16.,   0.],
        [  0.,  18.,  20.,  22.,  24.,   0.],
        [  0.,  26.,  28.,  30.,  32.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.]]])

In [411]:
np.allclose(my_convd(dat, kernel, stride=1, pad=0), fftconvolve(dat, kernel, 'valid'))

True

In [317]:
num_test = 1
for i in range(num_test):
    ndim = np.random.randint(1, 3)
    ndim=2
    shape = (3,3)#tuple(np.random.randint(1, 4) for i in range(ndim))
    k_shape = (2,2)#tuple(np.random.randint(1,1+dim) for dim in shape)
    dat = np.random.rand(*shape)
    kernel = np.random.rand(*k_shape)
    
    if not np.allclose(my_convd(dat, kernel, stride=1, pad=0), fftconvolve(dat, kernel, 'valid')):
        print dat.shape
        print kernel.shape
        print np.mean(np.absolute(my_convd(dat, kernel, stride=1, pad=0)-fftconvolve(dat, kernel, 'valid')))

(3, 3)
(2, 2)
0.0154491605382


In [358]:
shape = (3,3)#tuple(np.random.randint(1, 4) for i in range(ndim))
k_shape = (2,2)#tuple(np.random.randint(1,1+dim) for dim in shape)
dat = np.random.rand(*shape)
kernel = np.random.rand(*k_shape)

In [359]:
my_convd(dat, kernel, stride=1, pad=0)

array([[ 0.4056899 ,  0.4018525 ],
       [ 0.27214298,  0.19755556]])

In [360]:
fftconvolve(dat, kernel, 'valid')

array([[ 0.3693338 ,  0.35813452],
       [ 0.36373817,  0.31712687]])

In [361]:
convolve(dat,kernel,'valid')

array([[ 0.3693338 ,  0.35813452],
       [ 0.36373817,  0.31712687]])

In [362]:
dat

array([[ 0.46502953,  0.63891088,  0.41157319],
       [ 0.67468117,  0.75216321,  0.60159569],
       [ 0.39149867,  0.29911723,  0.04545069]])

In [363]:
kernel

array([[ 0.17166835,  0.01520908],
       [ 0.15552573,  0.28080544]])

In [365]:
np.sum(dat[:2,:2]*kernel)

0.40568990058238336

In [308]:
np.sum(dat[:2,1:3]*kernel)

0.54274133223270915

In [185]:
%%timeit
fftconvolve(dat, kernel, 'valid')

The slowest run took 4.17 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 84.4 µs per loop


In [52]:
outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*pad)/stride+1.
outshape = np.round(outshape).astype(int)

In [54]:
for x in product(*[stride*np.arange(i) for i in outshape]):
    print x

(0, 0)
(0, 2)
(2, 0)
(2, 2)


In [2]:
a = np.arange(10000).reshape((10,10,10,10))

In [6]:
%%timeit
a.reshape(a.shape[0], -1)

The slowest run took 16.76 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 366 ns per loop


In [7]:
b = np.arange(10000).reshape((10,10,10,10))
b = b.reshape(b.shape[0], -1)

In [8]:
%%timeit
b.reshape(b.shape[0], -1)

The slowest run took 16.30 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 345 ns per loop


In [14]:
a = np.float_(1).astype('float64')

In [13]:
a.astype('float64')

1.0

In [13]:
range(10, 0, -1)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [24]:
a = np.arange(640).reshape((10,4,4,4))

In [26]:
b = np.arange(640).reshape((10,4,4,4))

In [16]:
a[:1]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]])

In [21]:
a.shape

(10, 64)

In [27]:
a = a.reshape(a.shape[0], -1)
b = b.reshape(-1, b.shape[0])

In [28]:
b.shape

(64, 10)

In [29]:
a.shape

(10, 64)

In [30]:
np.allclose(b, a.T)

False

In [35]:
a.T.shape

(64, 10)

In [34]:
b.shape

(64, 10)

In [36]:
np.random.normal(0,1,(2,3))

array([[ 2.25709655, -2.46088041,  0.01490107],
       [-0.65288387, -1.10968923,  0.71995495]])